# Compute animationcodes of DB Trips

- load timetable from disk
- optionally apply realtime updates
- compute arrival and departure direction
- compute statuscodes

runs every 30 seconds
(in cron, register one job to run every minute and the other as well, but with 30s sleep at first)

## convenience functions

In [9]:
# chatgpt generiert lol
import datetime
def dateToDBDate(date: datetime.date) -> str:
    """
    Wandelt ein datetime.date Objekt in einen DB-Date-String "YYMMDD" um.
    """
    return date.strftime("%y%m%d")


def datetimeToDBDatetime(dt: datetime.datetime) -> str:
    """
    Wandelt ein datetime.datetime Objekt in einen DB-Datetime-String "YYMMDDHHMM" um.
    """
    return dt.strftime("%y%m%d%H%M")


def DBDatetimeToDatetime(dbDate: str) -> datetime.datetime:
    """
    Wandelt einen DB-Datetime-String "YYMMDDHHMM" in ein datetime.datetime Objekt um.
    """
    return datetime.datetime.strptime(dbDate, "%y%m%d%H%M")


def DBDateToDate(dbDate: str) -> datetime.date:
    """
    Wandelt einen DB-Date-String "YYMMDD" in ein datetime.date Objekt um.
    """
    return datetime.datetime.strptime(dbDate, "%y%m%d").date()
    
       
print(dateToDBDate(datetime.date(2025, 8, 10)))
print(datetimeToDBDatetime(datetime.datetime(2025, 8, 10, 12, 22)))
print(DBDateToDate("250810"))
print(DBDatetimeToDatetime("2508101222"))

250810
2508101222
2025-08-10
2025-08-10 12:22:00


## load timetable from disk

In [10]:
import pandas as pd

df_timetable = pd.read_csv('db_timetable.csv', dtype=str)


print(df_timetable.head(5))

  category line arrival_dbtime  \
0        S    3     2508100453   
1        S    2     2508100429   
2        S    2            NaN   
3        S    1     2508100403   
4       RB   68            NaN   

                                        arrival_path departure_dbtime  \
0  Ludwigshafen(Rh)Hbf|Ludwigshafen(Rhein) Mitte|...       2508100455   
1  Mannheim Hbf|Mannheim-Friedrichsfeld Süd|Heide...              NaN   
2                                                NaN       2508100432   
3  Karlsruhe Hbf|Karlsruhe-Durlach|Weingarten(Bad...       2508100413   
4                                                NaN       2508100520   

                                      departure_path  
0  Heidelberg-Kirchheim/Rohrbach|St Ilgen-Sandhau...  
1                                                NaN  
2  Heidelberg-Pfaffengrund/Wieblingen|Mannheim-Fr...  
3  Heidelberg-Pfaffengrund/Wieblingen|Mannheim-Fr...  
4  Heidelberg-Pfaffengrund/Wieblingen|Neu-Edingen...  


# OPTIONAL IMPROVEMENT: APPLY REALTIME UPDATES HERE

## add artificial departure / arrival times
if a trip starts/ends in Heidelberg Hbf, the arrival / departure properties are missing. To allow correct animation and not make the train spawn somewhere on the tracks, it is assumed that trains stand 1 minute in the station before departuring / after arriving. Therefore, one minute of standing time is artificially added to rows that don't have an arrival / departure.

In [11]:
import datetime

def fillEmptyArrivals(timetable_row):
    if pd.isna(timetable_row['arrival_dbtime']):
        departure_datetime = DBDatetimeToDatetime(timetable_row['departure_dbtime'])
        artificial_arrival_datetime = departure_datetime - datetime.timedelta(minutes=1)
        timetable_row['arrival_dbtime'] = datetimeToDBDatetime(artificial_arrival_datetime)
    return timetable_row

def fillEmptyDepartures(timetable_row):
    if pd.isna(timetable_row['departure_dbtime']):
        arrival_datetime = DBDatetimeToDatetime(timetable_row['arrival_dbtime'])
        artificial_departure_datetime = arrival_datetime + datetime.timedelta(minutes=1)
        timetable_row['departure_dbtime'] = datetimeToDBDatetime(artificial_departure_datetime)
    return timetable_row
        
df_timetable = df_timetable.apply(func=fillEmptyArrivals, axis=1)
df_timetable = df_timetable.apply(func=fillEmptyDepartures, axis=1)

print(df_timetable.head(5))
        
        

  category line arrival_dbtime  \
0        S    3     2508100453   
1        S    2     2508100429   
2        S    2     2508100431   
3        S    1     2508100403   
4       RB   68     2508100519   

                                        arrival_path departure_dbtime  \
0  Ludwigshafen(Rh)Hbf|Ludwigshafen(Rhein) Mitte|...       2508100455   
1  Mannheim Hbf|Mannheim-Friedrichsfeld Süd|Heide...       2508100430   
2                                                NaN       2508100432   
3  Karlsruhe Hbf|Karlsruhe-Durlach|Weingarten(Bad...       2508100413   
4                                                NaN       2508100520   

                                      departure_path  
0  Heidelberg-Kirchheim/Rohrbach|St Ilgen-Sandhau...  
1                                                NaN  
2  Heidelberg-Pfaffengrund/Wieblingen|Mannheim-Fr...  
3  Heidelberg-Pfaffengrund/Wieblingen|Mannheim-Fr...  
4  Heidelberg-Pfaffengrund/Wieblingen|Neu-Edingen...  


## compute arrival / departure direction
next, we want to compute the direction, from which a train is coming / to which a train is departuring. In the current model there are 4 possible directions that will be identified as north, south, west and east depending on which stations the respective planned path contains. There a multiple marker stations that are searched for in the path to determine the direction.


In [12]:
north_marker_stations = ['Neu-Edingen/Friedrichsfeld', 'Weinheim(Bergstr)Hbf', 'Bensheim', 'Darmstadt Hbf'] # RB68, FV nach Darmstadt
east_marker_stations = ['Heidelberg-Altstadt', 'Neckargemünd', 'Eberbach', 'Meckesheim'] # S1, S2, S5, S51, RE10a, RE10b
south_marker_stations = ['Wiesloch-Walldorf', 'Bruchsal', 'Karlsruhe Hbf', 'Vaihingen(Enz)','Stuttgart Hbf', 'Esslingen(Neckar)', 'Ulm Hbf'] # S3, S4, RE71, RE73, FV nach Karlsruhe, Stuttgart, Ulm 
west_marker_stations = ['Mannheim Hbf', 'Ludwigshafen(Rhein) Mitte','Ludwigshafen(Rh)Hbf'] # S1, S2, S3, S4, RE10a, RE10b, FV über Mannheim nach Wiesbaden / Mainz / Frankfurt 




def compute_path_direction(path: str):
    # check if path contains any of the marker stations
    if pd.isna(path):
        return "DEPOT"
    if any(station in path for station in north_marker_stations):
        return "north"
    if any(station in path for station in east_marker_stations):
        return "east"
    if any(station in path for station in west_marker_stations):
        return "west"
    if any(station in path for station in south_marker_stations):
        return "south"
 
   
df_timetable['arrival_direction'] = df_timetable['arrival_path'].map(compute_path_direction)
#df_timetable = df_timetable.drop('arrival_path', axis=1)

df_timetable['departure_direction'] = df_timetable['departure_path'].map(compute_path_direction)
#df_timetable = df_timetable.drop('departure_path', axis=1)

df_timetable.to_csv('temp.csv')

## compute animation class
trips are classified into two animation classes, depending on which station stops have to be animated.  
Class NV: S, RB - stop at Hbf, Kirchheim/Rohrbach, Weststadt/Südstadt, Pfaffengrund/Wieblingen, Neu-Edingen/Friedrichsfeld  
Class FV: everything else (RE, FLX, IC, ICE, NJ, RJ, RJX, TGV) - only stop at Hbf  
The class will be included in the statuscode
each class gets their own statuscode_led_mapping, so that the statuscode can be computed in ticks after / before HD Hbf, and depending on the class the statuscodes mean different positions to account for stops


## compute animation color
each category receives a (not unique) color, which is used to display the respective trip.
The color will later be included into the animationcode in two ways, the original color and a dimmed color for the trail

## compute animationcodes


## save to disk